# Strong business tester

This notebook tries to test if a business is strong based on some financial parameters:

* Strong free cash flow
* Strong balance sheet

In [1]:
import matplotlib.pyplot as plt

In [2]:
from yahooquery import Ticker

In [3]:
ticker = Ticker('TSLA')

## Cash flow analysis

In [4]:
cash_flow = ticker.cash_flow(frequency='Annual')

In [5]:
type(cash_flow)

pandas.core.frame.DataFrame

In [6]:
cash_flow.columns

Index(['asOfDate', 'periodType', 'currencyCode', 'AssetImpairmentCharge',
       'BeginningCashPosition', 'CapitalExpenditure',
       'CapitalExpenditureReported',
       'CashFlowFromContinuingFinancingActivities',
       'CashFlowFromContinuingInvestingActivities',
       'CashFlowFromContinuingOperatingActivities',
       'ChangeInCashSupplementalAsReported', 'ChangeInInventory',
       'ChangeInOtherCurrentAssets', 'ChangeInOtherCurrentLiabilities',
       'ChangeInOtherWorkingCapital', 'ChangeInPayablesAndAccruedExpense',
       'ChangeInPrepaidAssets', 'ChangeInReceivables',
       'ChangeInWorkingCapital', 'ChangesInAccountReceivables',
       'ChangesInCash', 'CommonStockIssuance', 'Depreciation',
       'DepreciationAmortizationDepletion', 'DepreciationAndAmortization',
       'EffectOfExchangeRateChanges', 'EndCashPosition', 'FinancingCashFlow',
       'FreeCashFlow', 'GainLossOnSaleOfPPE', 'IncomeTaxPaidSupplementalData',
       'InterestPaidSupplementalData', 'InvestingCas

In [7]:
cash_flow[['asOfDate', 'FreeCashFlow']].style.format({
    'FreeCashFlow': '{:,.2f}'.format,
})

,asOfDate,FreeCashFlow
symbol,,
TSLA,2019-12-31 00:00:00,"968,000,000.00"
TSLA,2020-12-31 00:00:00,"2,701,000,000.00"
TSLA,2021-12-31 00:00:00,"3,483,000,000.00"
TSLA,2022-12-31 00:00:00,"7,552,000,000.00"
TSLA,2022-12-31 00:00:00,"7,552,000,000.00"
TSLA,2023-03-31 00:00:00,"5,778,000,000.00"


### Has strong free cash flow?

In [8]:
free_cash_flow = cash_flow[['asOfDate', 'FreeCashFlow']]

In [9]:
free_cash_flow = free_cash_flow.set_index('asOfDate')
fcf = free_cash_flow.to_dict()['FreeCashFlow']
fcf

{Timestamp('2019-12-31 00:00:00'): 968000000.0,
 Timestamp('2020-12-31 00:00:00'): 2701000000.0,
 Timestamp('2021-12-31 00:00:00'): 3483000000.0,
 Timestamp('2022-12-31 00:00:00'): 7552000000.0,
 Timestamp('2023-03-31 00:00:00'): 5778000000.0}

In [10]:
# Is FCF positive in the past 4-5 years?
has_good_fcf = all([v > 0 for v in fcf.values()])
has_good_fcf

True

## Has strong business?

In [11]:
def non_decreasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))

def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def monotonic(L):
    return non_decreasing(L) or non_increasing(L)

def strictly_increasing(L):
    return all(x<y for x, y in zip(L, L[1:]))

def strictly_decreasing(L):
    return all(x>y for x, y in zip(L, L[1:]))

def strictly_monotonic(L):
    return strictly_increasing(L) or strictly_decreasing(L)

In [12]:
def has_consecutive_positive_fcf(ticker):
    cash_flow = ticker.cash_flow(frequency='Annual')
    free_cash_flow = cash_flow[['asOfDate', 'FreeCashFlow']]
    free_cash_flow = free_cash_flow.set_index('asOfDate')
    fcf = free_cash_flow.to_dict()['FreeCashFlow']
    return all([v > 0 for v in fcf.values()])

In [13]:
def has_consistently_low_debt_ratios(debt_equity_ratio_values):
    return all([v < 0.3 for v in debt_equity_ratio_values])


def has_strong_balance_sheet(ticker):
    balance_sheet = ticker.balance_sheet(frequency='Quarterly')
    balance_sheet['TotalDebt/CommonStockEquity']=balance_sheet['TotalDebt']/balance_sheet['CommonStockEquity']
    debt_equity_ratio = balance_sheet[['asOfDate', 'TotalDebt/CommonStockEquity']]
    debt_equity_ratio = debt_equity_ratio.set_index('asOfDate').to_dict()['TotalDebt/CommonStockEquity']
    debt_equity_ratio_values = list(debt_equity_ratio.values())

    return has_consistently_low_debt_ratios(debt_equity_ratio_values) and strictly_decreasing(debt_equity_ratio_values)

In [14]:
def has_strong_business(ticker):
    if not has_consecutive_positive_fcf(ticker):
        print(f"{ticker} doesn't have consecutive positive fcf")
        return False
    
    if not has_strong_balance_sheet(ticker):
        print(f"{ticker} doesn't have strong balance sheet")
        return False
    
    return True

In [15]:
has_strong_business(Ticker('TSLA'))

True

In [16]:
has_strong_business(Ticker('LCID'))

<yahooquery.ticker.Ticker object at 0x11c35e0d0> doesn't have consecutive positive fcf


False

In [17]:
has_strong_business(Ticker('AMZN'))

<yahooquery.ticker.Ticker object at 0x11c352490> doesn't have consecutive positive fcf


False

In [18]:
has_strong_business(Ticker('BABA'))

<yahooquery.ticker.Ticker object at 0x11c358a10> doesn't have strong balance sheet


False